In [ ]:
import pandas as pd
import pickle
import os

import plotly.express as px
import plotly.graph_objects as go
import torch

In [ ]:
mount_point = "/mnt/raid/C1_ML_Analysis/"



In [ ]:
folds_dir = "test_output/fetal_biometry/efw/efw_2025-10-31_train_with_uid_folds"
pred_fold0 = pickle.load(open(os.path.join(mount_point, folds_dir, 'fold0/prediction.pkl'), 'rb'))
df_pred_fold0 = pd.DataFrame({'uid': pred_fold0['id'], 'efw_pred': pred_fold0['efw_pred']})
df_fold0 = pd.read_csv(os.path.join(mount_point, 'CSV_files/efw_2025-10-31_train_with_uid_fold0_test.csv'))

pred_fold1 = pickle.load(open(os.path.join(mount_point, folds_dir, 'fold1/prediction.pkl'), 'rb'))
df_pred_fold1 = pd.DataFrame({'uid': pred_fold1['id'], 'efw_pred': pred_fold1['efw_pred']})
df_fold1 = pd.read_csv(os.path.join(mount_point, 'CSV_files/efw_2025-10-31_train_with_uid_fold1_test.csv'))

pred_fold2 = pickle.load(open(os.path.join(mount_point, folds_dir, 'fold2/prediction.pkl'), 'rb'))
df_pred_fold2 = pd.DataFrame({'uid': pred_fold2['id'], 'efw_pred': pred_fold2['efw_pred']})
df_fold2 = pd.read_csv(os.path.join(mount_point, 'CSV_files/efw_2025-10-31_train_with_uid_fold2_test.csv'))

pred_fold3 = pickle.load(open(os.path.join(mount_point, folds_dir, 'fold3/prediction.pkl'), 'rb'))
df_pred_fold3 = pd.DataFrame({'uid': pred_fold3['id'], 'efw_pred': pred_fold3['efw_pred']})
df_fold3 = pd.read_csv(os.path.join(mount_point, 'CSV_files/efw_2025-10-31_train_with_uid_fold3_test.csv'))

pred_fold4 = pickle.load(open(os.path.join(mount_point, folds_dir, 'fold4/prediction.pkl'), 'rb'))
df_pred_fold4 = pd.DataFrame({'uid': pred_fold4['id'], 'efw_pred': pred_fold4['efw_pred']})
df_fold4 = pd.read_csv(os.path.join(mount_point, 'CSV_files/efw_2025-10-31_train_with_uid_fold4_test.csv'))

df_folds = pd.concat([df_fold0, df_fold1, df_fold2, df_fold3, df_fold4], axis=0).reset_index(drop=True)
df_pred_folds = pd.concat([df_pred_fold0, df_pred_fold1, df_pred_fold2, df_pred_fold3, df_pred_fold4], axis=0).reset_index(drop=True)

df_pred_folds['efw_pred'] = df_pred_folds['efw_pred'].astype(float)*1000
df_pred_folds = df_pred_folds.merge(df_folds, on='uid')

In [ ]:
df_pred_folds['efw_error'] = df_pred_folds['efw_pred'] - df_pred_folds['efw_gt']
df_pred_folds['abs_efw_error'] = df_pred_folds['efw_error'].abs()
df_pred_folds['efw_bin'] = pd.cut(df_pred_folds['efw_pred'], bins=[0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000], labels=['0-500','500-1k','1k-1.5k','1.5k-2k','2k-2.5k','2.5k-3k','3k-3.5k','3.5k-4k','4k-4.5k','4.5k-5k'])

In [ ]:
px.violin(df_pred_folds[['abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution - Train Folds').show()

In [ ]:
df_min_error = df_pred_folds.loc[df_pred_folds.groupby('study_id')['abs_efw_error'].idxmin()]


In [ ]:
prediction = pickle.load(open(os.path.join(mount_point, 'test_output/fetal_biometry/efw/v2.1/efw_2025-11-12_test/prediction.pkl'), 'rb'))

max_scores_frames = [torch.max(s).item() for s in prediction['scores_frames']]

In [ ]:
prediction.keys()

In [ ]:

df_test = pd.read_csv('/mnt/raid/C1_ML_Analysis/CSV_files/efw_2025-11-12_test.csv')
df_pred = pd.DataFrame({'study_id': prediction['id'], 'efw_pred': prediction['efw_pred'], 'max_scores_frames': max_scores_frames})
df_pred['efw_pred'] = df_pred['efw_pred'].astype(float)*1000
df_pred = df_pred.merge(df_test, on='study_id')

In [ ]:
df_test.columns

In [ ]:

df_pred['efw_error'] = df_pred['efw_pred'] - df_pred['efw_gt']
df_pred['abs_efw_error'] = df_pred['efw_error'].abs()
df_pred['efw_bin'] = pd.cut(df_pred['efw_pred'], bins=[0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000], labels=['0-500','500-1k','1k-1.5k','1.5k-2k','2k-2.5k','2.5k-3k','3k-3.5k','3.5k-4k','4k-4.5k','4.5k-5k'])

In [ ]:
px.violin(df_pred[['abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution').show()

In [ ]:
px.violin(df_pred.query('max_scores_frames > 0.5')[['abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution').show()

In [ ]:
df_pred['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_pred.query('max_scores_frames > 0.5')['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_pred.query('max_scores_frames > 0.75')['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
print(df_pred['study_id'].drop_duplicates().shape, df_pred.query('max_scores_frames > 0.5')['study_id'].drop_duplicates().shape)

In [ ]:
df_pred.query('study_id == "000019699255_20211022"')['file_path'].values

In [ ]:
df_pred['abs_efw_error'].hist(bins=50)

In [ ]:
df_pred.query('study_id == "000014393698_20231102"').index


In [ ]:
prediction_sweep = pickle.load(open(os.path.join(mount_point, 'test_output/fetal_biometry/efw/v2.1/efw_2025-11-12_test/prediction_sweep.pkl'), 'rb'))
max_scores_sweep = [torch.max(s).item() for s in prediction_sweep['scores_sweeps']]

In [ ]:
df_prediction_sweep = pd.DataFrame({'file_path': prediction_sweep['file_path'], 'study_id': prediction_sweep['id'], 'efw_pred': prediction_sweep['efw_pred']*1000, 'efw_gt': prediction_sweep['efw_gt']*1000, 'max_scores_frames': max_scores_sweep})
df_prediction_sweep['efw_error'] = df_prediction_sweep['efw_pred'] - df_prediction_sweep['efw_gt']
df_prediction_sweep['abs_efw_error'] = df_prediction_sweep['efw_error'].abs()
df_prediction_sweep['efw_bin'] = pd.cut(df_prediction_sweep['efw_pred'], bins=[0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000], labels=['0-500','500-1000','1000-1500','1500-2000','2000-2500','2500-3000','3000-3500','3500-4000','4000-4500','4500-5000'])

df_prediction_sweep = df_prediction_sweep.merge(df_test[['study_id', 'dataset']], on='study_id')

In [ ]:
px.violin(df_prediction_sweep[['study_id', 'abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution - DXA Test Set').show()

In [ ]:
px.violin(df_prediction_sweep.query('max_scores_frames > 0.5')[['study_id', 'abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution - DXA Test Set').show()

In [ ]:
px.violin(df_prediction_sweep.query('max_scores_frames > 0.8')[['study_id', 'abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution - DXA Test Set').show()

In [ ]:
df_prediction_sweep['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_prediction_sweep.query('max_scores_frames > 0.8')['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_max_score = df_prediction_sweep.loc[df_prediction_sweep.groupby('study_id')['max_scores_frames'].idxmax()]


In [ ]:
px.violin(df_max_score[['study_id', 'abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution - DXA Test Set').show()

In [ ]:
px.violin(df_max_score.query('max_scores_frames > 0.8')[['study_id', 'abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution - DXA Test Set').show()

In [ ]:
list(df_max_score.query('max_scores_frames > 0.8 and abs_efw_error > 1000')['file_path'])

In [ ]:
df_max_score['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_max_score.query('max_scores_frames > 0.8')['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
prediction['scores_frames'][83]

In [ ]:
list(df_min_error.query('abs_efw_error > 750')['study_id'].drop_duplicates().values)

In [ ]:
df_min_error['abs_efw_error'].hist(bins=50)

In [ ]:
prediction_dxa = pickle.load(open(os.path.join(mount_point, 'test_output/fetal_biometry/efw/v2.1/efw_2025-11-04_dxa_test/prediction.pkl'), 'rb'))
max_scores_frames = [torch.max(s).item() for s in prediction_dxa['scores_frames']]

In [ ]:
prediction_dxa.keys()

In [ ]:
df_test_dxa = pd.read_csv('/mnt/raid/C1_ML_Analysis/CSV_files/efw_2025-11-04_dxa_test.csv')

df_pred_dxa = pd.DataFrame({'study_id': prediction_dxa['id'], 'efw_pred': prediction_dxa['efw_pred'], 'max_scores_frames': max_scores_frames})
df_pred_dxa['efw_pred'] = df_pred_dxa['efw_pred'].astype(float)*1000
df_pred_dxa = df_pred_dxa.merge(df_test_dxa, on='study_id')

In [ ]:
df_pred_dxa['efw_error'] = df_pred_dxa['efw_pred'] - df_pred_dxa['efw_gt']
df_pred_dxa['abs_efw_error'] = df_pred_dxa['efw_error'].abs()
df_pred_dxa['efw_bin'] = pd.cut(df_pred_dxa['efw_pred'], bins=[0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000], labels=['0-500','500-1000','1000-1500','1500-2000','2000-2500','2500-3000','3000-3500','3500-4000','4000-4500','4500-5000'])

In [ ]:
px.violin(df_pred_dxa[['study_id', 'abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', box=True, points='all', title='EFW Absolute Error Distribution - DXA Test Set').show()

In [ ]:
px.violin(df_pred_dxa.query('max_scores_frames > 0.5')[['study_id', 'abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', box=True, points='all', title='EFW Absolute Error Distribution - DXA Test Set').show()

In [ ]:
df_pred_dxa['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_pred_dxa.query('max_scores_frames > 0.5')['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_pred_dxa.query('max_scores_frames > 0.75')['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
prediction_sweep_dxa = pickle.load(open(os.path.join(mount_point, 'test_output/fetal_biometry/efw/v2.1/efw_2025-11-04_dxa_test/prediction_sweep.pkl'), 'rb'))
max_scores_sweep_dxa = [torch.max(s).item() for s in prediction_sweep_dxa['scores_sweeps']]

In [ ]:
prediction_sweep_dxa.keys()

In [ ]:
df_pred_sweep_dxa = pd.DataFrame({'file_path': prediction_sweep_dxa['file_path'], 'study_id': prediction_sweep_dxa['id'], 'efw_pred': prediction_sweep_dxa['efw_pred']*1000, 'efw_gt': prediction_sweep_dxa['efw_gt']*1000, 'max_scores_frames': max_scores_sweep_dxa})
df_pred_sweep_dxa['efw_error'] = df_pred_sweep_dxa['efw_pred'] - df_pred_sweep_dxa['efw_gt']
df_pred_sweep_dxa['abs_efw_error'] = df_pred_sweep_dxa['efw_error'].abs()
df_pred_sweep_dxa['efw_bin'] = pd.cut(df_pred_sweep_dxa['efw_pred'], bins=[0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000], labels=['0-500','500-1000','1000-1500','1500-2000','2000-2500','2500-3000','3000-3500','3500-4000','4000-4500','4500-5000'])



In [ ]:
px.violin(df_pred_sweep_dxa[['study_id', 'abs_efw_error', 'efw_bin']].drop_duplicates(), y='abs_efw_error', x='efw_bin', box=True, points='all', title='EFW Absolute Error Distribution - DXA Test Set').show()

In [ ]:
px.violin(df_pred_sweep_dxa.query('max_scores_frames > 0.8')[['study_id', 'abs_efw_error', 'efw_bin']].drop_duplicates(), y='abs_efw_error', x='efw_bin', box=True, points='all', title='EFW Absolute Error Distribution - DXA Test Set').show()

In [ ]:
df_pred_sweep_dxa['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_pred_sweep_dxa.query('max_scores_frames > 0.5')['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_pred_sweep_dxa.query('max_scores_frames > 0.8')['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_pred_sweep_dxa.query('max_scores_frames > 0.8 and abs_efw_error > 750')[['study_id', 'file_path']].drop_duplicates().values

In [ ]:

df_pred_sweep_dxa.query("study_id == 'DXA-025-0001_20230206-bfly-app-expert'")


In [ ]:
df_pred_sweep_dxa.iloc[11]['file_path']

In [ ]:
prediction_sweep_dxa['scores_sweeps']

In [ ]:
df_max_score_dxa = df_pred_sweep_dxa.loc[df_pred_sweep_dxa.groupby('study_id')['max_scores_frames'].idxmax()]


In [ ]:
px.violin(df_max_score_dxa[['study_id', 'abs_efw_error', 'efw_bin']].drop_duplicates(), y='abs_efw_error', x='efw_bin', box=True, points='all', title='EFW Absolute Error Distribution - DXA Test Set').show()

In [ ]:
px.violin(df_max_score_dxa.query('max_scores_frames > 0.8')[['study_id', 'abs_efw_error', 'efw_bin']].drop_duplicates(), y='abs_efw_error', x='efw_bin', box=True, points='all', title='EFW Absolute Error Distribution - DXA Test Set').show()

In [ ]:
df_max_score_dxa['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_max_score_dxa.query('max_scores_frames > 0.5')['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_max_score_dxa.query('max_scores_frames > 0.8')['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_max_score_dxa.query('max_scores_frames > 0.9')['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])